# Option 1 example output

In [3]:
import pandas as pd
import pm4py

from pm4py.objects.dcr.extended.obj import ExtendedDcrGraph
graph = ExtendedDcrGraph()

events = [
    "A", "B", "C", "D", "E", "F",  "N1", "N2", "N3"
]


for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)



#graph.nestedgroups["N1"] =  {"B", "N2"} 
#graph.nestedgroups["N2"] =  {"C", "D"} 
#graph.nestedgroups["N3"] =  {"E", "F"} 

graph.superActivities = {"N1" : {"B", "N2"}, "N2" : {"C", "D"}, "N3" : {"E", "F"}}

'''
for group, events in graph.nestedgroups.items():
    for e in events:
        graph.nestedgroups_map[e] = group
'''

# No corrosponding codeblock


graph.marking.included.remove("B")
graph.marking.pending.add("D")

# graph.conditions["N1"] = {"A"}
# graph.conditions["N3"] = {"N1"}
print("Cond")
graph.conditions = {"N1" : {"A"}, "N3" : {"N1"}}

# graph.responses["N2"] = {"N3"}
print("\nRes")
graph.responses = {"N2" : {"N3"}}

# graph.includes["N3"] = {"B"}
print("\nInc")

graph.includes = {"N3" : {"B"}}

# graph.excludes["C"] = {"N2"}
print("\nExc")
graph.excludes = {"C" : {"N2"}}


Cond
{'N1': {'A'}, 'N3': {'N1'}}
{'N2': {'A'}, 'B': {'A'}, 'E': {'N2', 'B'}, 'F': {'N2', 'B'}}
{'B': {'A'}, 'E': {'C', 'D', 'B'}, 'F': {'C', 'D', 'B'}, 'C': {'A'}, 'D': {'A'}}

Res
{'N2': {'N3'}}

Inc
{'N3': {'B'}}

Exc
{'C': {'N2'}}


In [4]:
import pandas as pd
from io import StringIO
import pm4py
from pm4py.objects.dcr.extended.semantics import ExtendedSemantics    # Should be HierarchicalDcrSemantics instead!

semantics = ExtendedSemantics()

errors = 0

expected_enabled = ["A"] # As seem in the simulation
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")        


semantics.execute(graph, "A")
expected_enabled = ["A", "C", "D"]
print("1th execution (Event A)")
print("Expected enabled", expected_enabled)
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                print("event", e)
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")         


semantics.execute(graph, "C")
expected_enabled = ["A", "E", "F"]
print("2nd execution (Event C)")
print("Expected enabled", expected_enabled)
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")                


semantics.execute(graph, "E")
expected_enabled = ["A", "B"]
print("3rd execution (Event E)")
print("Expected enabled", expected_enabled)
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                print("event", e)
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")                


semantics.execute(graph, "B")
expected_enabled = ["A", "B", "E", "F"]
print("4th execution (Event B)")
print("Expected enabled", expected_enabled)
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
        
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")    
            
semantics.execute(graph, "F")
expected_enabled = ["A", "B", "E", "F"]
print("5th execution (Event F)")
print("Expected enabled", expected_enabled)
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if not semantics.is_accepting(graph):
        errors+=1
        raise Exception("Graph should not be accepting")                

if errors == 0:
      print("\nThe implementation passes all tests")
else:
      print("\nFound", errors)



1th execution (Event A)
Expected enabled ['A', 'C', 'D']
2nd execution (Event C)
Expected enabled ['A', 'E', 'F']
3rd execution (Event E)
Expected enabled ['A', 'B']
4th execution (Event B)
Expected enabled ['A', 'B', 'E', 'F']
5th execution (Event E)
Expected enabled ['A', 'B', 'E', 'F']

The implementation passes all tests
